# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [30]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [31]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,papatowai,-46.5619,169.4708,8.03,93,100,1.32,NZ,1722197007
1,1,alo,6.5486,4.2686,24.70,87,100,5.30,NG,1722197008
2,2,olinda,-8.0089,-34.8553,26.85,74,40,5.66,BR,1722197010
3,3,hamilton,39.1834,-84.5333,25.80,89,75,6.17,US,1722197011
4,4,port-aux-francais,-49.3500,70.2167,0.48,65,97,14.82,TF,1722197012


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [32]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    s = "Humidity"
)

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [33]:
# Narrow down cities that fit criteria and drop any results with null values
# I like cool places in the summer, where it's not too humid
pleasant_df = city_data_df.loc[
        city_data_df["Max Temp"] < 18].loc[
        city_data_df["Max Temp"] > 10].loc[
        city_data_df["Humidity"] < 66]
# Drop any rows with null values
pleasant_df = pleasant_df.dropna()

# Display sample data
pleasant_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,mpongwe,-13.5091,28.1550,16.02,17,0,2.30,ZM,1722197022
16,16,ilulissat,69.2167,-51.1000,10.01,57,100,5.66,GL,1722197027
55,55,anadyr,64.7500,177.4833,11.28,43,0,2.00,RU,1722197075
62,62,mount isa,-20.7333,139.5000,10.87,50,0,3.09,AU,1722197084
71,71,iringa,-7.7667,35.7000,12.07,63,8,2.53,TZ,1722197095
77,77,margaret river,-33.9500,115.0667,15.96,54,100,7.44,AU,1722197102
79,79,general acha,-37.3770,-64.6043,11.23,20,0,4.59,AR,1722197104
99,99,karratha,-20.7377,116.8463,17.00,29,1,3.48,AU,1722197129
136,136,kitimat,54.0524,-128.6534,16.21,61,100,1.79,CA,1722197174
145,145,san jose de jachal,-30.2406,-68.7469,11.14,39,13,3.50,AR,1722197185


### Step 3: Create a new DataFrame called `hotel_df`.

In [34]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pleasant_df.copy()
hotel_df = hotel_df.loc[:, ["City", "Country", "Lat", "Lng", "Humidity"] ]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
12,mpongwe,ZM,-13.5091,28.1550,17,
16,ilulissat,GL,69.2167,-51.1000,57,
55,anadyr,RU,64.7500,177.4833,43,
62,mount isa,AU,-20.7333,139.5000,50,
71,iringa,TZ,-7.7667,35.7000,63,
77,margaret river,AU,-33.9500,115.0667,54,
79,general acha,AR,-37.3770,-64.6043,20,
99,karratha,AU,-20.7377,116.8463,29,
136,kitimat,CA,54.0524,-128.6534,61,
145,san jose de jachal,AR,-30.2406,-68.7469,39,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [37]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
params = {
    "limit":limit,
    "apiKey":geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    params["filter"] = filters
    params["bias"] = bias

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mpongwe - nearest hotel: No hotel found
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
anadyr - nearest hotel: Гостевой дом
mount isa - nearest hotel: Ibis Styles
iringa - nearest hotel: Zakinn
margaret river - nearest hotel: Margaret River Hotel
general acha - nearest hotel: Hotel Patagonia
karratha - nearest hotel: Karratha International Hotel
kitimat - nearest hotel: MStar Hotel
san jose de jachal - nearest hotel: San Martin
minas de marcona - nearest hotel: No hotel found
cervantes - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
psyzh - nearest hotel: No hotel found
ust-nera - nearest hotel: гостиница "Солнечная"
susuman - nearest hotel: No hotel found
east london - nearest hotel: No hotel found
meadow lakes - nearest hotel: Best Western Lake Lucille Inn
talnakh - nearest hotel: Талнах
liwale - nearest hotel: No hotel found
murghob - nearest hotel: Pamir Hotel
whitehorse - nearest hotel: Town & Mountain Hotel
kozlovka 

,City,Country,Lat,Lng,Humidity,Hotel Name
12,mpongwe,ZM,-13.5091,28.1550,17,No hotel found
16,ilulissat,GL,69.2167,-51.1000,57,Best Western Plus Hotel Ilulissat
55,anadyr,RU,64.7500,177.4833,43,Гостевой дом
62,mount isa,AU,-20.7333,139.5000,50,Ibis Styles
71,iringa,TZ,-7.7667,35.7000,63,Zakinn
77,margaret river,AU,-33.9500,115.0667,54,Margaret River Hotel
79,general acha,AR,-37.3770,-64.6043,20,Hotel Patagonia
99,karratha,AU,-20.7377,116.8463,29,Karratha International Hotel
136,kitimat,CA,54.0524,-128.6534,61,MStar Hotel
145,san jose de jachal,AR,-30.2406,-68.7469,39,San Martin


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [42]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    hover_cols = ["Country", "Hotel Name"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Hotel Name)